In [137]:
import numpy as np

In [138]:
from tensorflow.keras import datasets

(X_tn0, y_tn0), (X_te0, y_te0) = datasets.mnist.load_data()

In [139]:
input = X_tn0[0]
input.shape

(28, 28)

In [140]:
target = np.array(y_tn0[0]).reshape(1, -1)
target

array([[5]], dtype=uint8)

In [141]:
# 비용 함수 클래스, 비용 함수의 종류에 따라 다른 클래스 내 함수를 사용한다.
class cost_function:
  # 예측값
  predict = []
  # 타겟값
  target = []
  # 비용 함수값
  error_cost = []

  # 오차 제곱합
  def errer_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    self.error_cost = np.sum(0.5*((predict - target)**2))
    return self.error_cost

  # 오차 제곱합 미분 함수
  def diff_error_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    return self.predict - self.target

In [142]:
# 활성화 함수
class activation_function:
  # 시그모이드 함수
  def sigmoid(self, x):
    return 1 / (1+np.e**-x)
  
  # 시그모이드 함수의 미분 함수
  def sigmoid_diff(self, x):  
    return self.sigmoid(x) * (1 - self.sigmoid(x))

In [143]:
class PADDING:

  def padding(self, input, padding_size):
    """
    input : 입력 데이터
    padding_size : padding 연산 후의 데이터 크기
    """
    if(padding_size == 0):
      return input

    padding_matrix = np.zeros((input.shape[0] + padding_size, input.shape[1] + padding_size))

    for i in range(input.shape[0]):
      for j in range(input.shape[1]):
        padding_matrix[i + int(padding_size / 2)][j + int(padding_size / 2)] = input[i][j]
    
    return padding_matrix


In [144]:
class POOLING:
  pad = PADDING()

  pool_result = []

  def max_pooling(self, input, pooling_size):
    """
    input : 입력 데이터
    pooling_size : pooling size
    """
    self.pool_result = []
    
    pooling_matrix = self.pad.padding(input, (input.shape[0] % pooling_size))

    for col in range(0, pooling_matrix.shape[0], pooling_size):
      for row in range(0, pooling_matrix.shape[1], pooling_size):
        pool_arr = []
        for pooling_col in range(pooling_size):
          for pooling_row in range(pooling_size):        
            pool_arr.append(pooling_matrix[pooling_col + col, pooling_row + row])
        self.pool_result.append(max(pool_arr))
    
    # 연산 결과를 크기에 맞게 바꿔준다.
    return np.array(self.pool_result).reshape(int(pooling_matrix.shape[0] / pooling_size), -1)

  # 풀링 결과와 매칭되는 입력값을 찾고, 해당 입력값과 연결되는 가중치에 대한 업데이트를 수행하기 위한 함수
  def matching_pooling_result(self, input, result):
    """
    input : pooling 수행 전 입력 데이터
    result : pooling 수행 후의 데이터
    """
    position_arr = []

    # 풀링 데이터의 반복
    for pooling_col in range(result.shape[0]):
      for pooling_row in range(result.shape[1]):
        # 전체 데이터에 대한 탐색
        for col in range(pooling_col, pooling_col + result.shape[0]):
          for row in range(pooling_row, pooling_row + result.shape[1] ):            
            # 풀링 결과와 동일한 값을 찾는다. 그 위치의 탐색
            if(result[pooling_col, pooling_row] == input[col, row]):
              data_position = [col, row]
        
        position_arr.append(data_position)

    return position_arr



In [145]:
class MLP:
  #가중치
  weight = []

  #편향값
  bias = []
    
  # 각 층별 노드들의 입력, 활성화 함수 연산 전의 값
  node_input = []

  # 각 층별 노드들의 출력
  node_output = []

  # 타겟, 목푯값
  target = []

  # 각 층의 델타 값의 저장
  delta = []

  # 비용 함수
  cost = cost_function()

  # 활성화 함수
  activation = activation_function()

  # 가중치 업데이트 크기
  weight_update_arr = []
  
  #순전파 계산
  def forward_cal(self, input, node_count):

    # 가중치의 임의 생성, node_count 개수만큼의 은닉(또는 출력) 노드가 존재한다.
    # (n,1) 개의 입력 값과 m 개의 노드 연결 (m, n) 크기의 가중치가 존재해야 한다.
    weight = np.random.rand(node_count, input.shape[0])

    #편향값의 임의 생성, 동일한 편향 값을 사용한다.
    bias = np.random.rand(1)

    # 노드 입력값의 저장 필요, 활성화 미분 함수의 값을 구하기 위해 필요
    self.node_input.append(input)

    #가중치와 노드 출력의 행렬곱연산, 편향값 덧셈
    hidden_input = weight @ input + bias
            
    #노드 입력과 활성화 함수 연산을 통한 노드 출력 계산
    # cnn 에선 0과 1 사이의 결과를 출력해야 하기 때문에 시그모이드 활성화 함수를 사용용
    output = self.activation.sigmoid(hidden_input)

    # 노드 출력의 저장
    self.node_output.append(output)

    # 가중치 값 저장
    self.weight.append(weight)

    # 편향 값 저장
    self.bias.append(bias)

    return output

  def cost_function(self, target):
    self.target = target
    return self.cost.errer_squared_sum(self.predict, target)

  # 비용 함수에 대한 delta 값 계산
  def cal_delta_result(self, predict, target):  

    #출력층 노드의 변화량에 대한 오차 함수의 변화량 계산
    delta = (self.cost.diff_error_squared_sum(predict, target) * self.activation.sigmoid_diff(self.node_output[-1]))

    self.delta.append(delta)

    return delta

  # 은닉층에 대한 delta 값 계산, 가중치 변화량이 누적된다.
  def cal_delta_hidden(self):

    # delta 값 호출
    delta = self.delta[-1]

    # 가중치와의 연산
    delta = self.weight[-len(self.delta)].T @ delta

    # 활성화 함수 미분 함수와의 연산
    delta = delta * self.activation.sigmoid_diff(self.node_input[-(len(self.delta))])

    self.delta.append(delta)

    return delta
  
  # 가중치 업데이트량 크기 계산
  def weight_update(self, learning_rate):
    # 연산의 편리함을 위해 뒤집어준다.
    self.delta = self.delta[::-1]

    for i in range(len(self.weight)):
      self.weight_update_arr.append(self.delta[i] @ self.node_output[i].T)

      self.weight[i] = self.weight[i] - self.weight_update_arr[i].T



In [146]:
class CNN:

  # 필터의 개수만큼 가중치가 존재
  cnn_weight = []
  
  # 패딩
  pad = PADDING()

  # 풀링
  pool = POOLING()

  # MLP
  mlp = MLP()  

  # 레이어 구현
  layer_result = []

  # 현재 레이어 
  layer = 0

  # delta 값
  delta = []

  # target 값
  target = []

  # delta 값 계산을 위한 cnn 연산
  def delta_cnn(self, weight):
    # 각 필터별 연산 결과를 저장 리스트
    filter_result_arr = []

    # 현재 델타값
    delta = self.delta[len(self.delta) - 1]

    # 필터의 개수
    filter_count = len(weight)

    # 필터의 모양
    filter_shape = int((len(delta) / filter_count) ** (1/2))

    # 필터의 개수만큼 반복
    for i in range(filter_count):
      # 현재 필터의 델타값
      filter_delta = delta[int(i * (len(delta) /filter_count)): int((i+1) * (len(delta) /filter_count))]
      
      # 델타값의 모양을 합성곱 연산에 맞게 바꿔준다.
      filter_delta = filter_delta.reshape(filter_shape, -1)

      # 합성곱 시 padding 을 수행했을 경우 역전파 계산에서도 padding 의 수행
      filter_delta = self.pad.padding(filter_delta, int(filter_shape))

      # 현재 필터의 가중치
      filter_weight = weight[i]

      # 합성곱 연산 결과의 저장
      result_arr = []

      # 합성곱 연산 수행
      for col in range(filter_shape):
        for row in range(filter_shape):
          result = []
          for w_col in range(filter_weight.shape[0]):
            for w_row in range(filter_weight.shape[1]):
              result.append(filter_delta[col + w_col, row + w_row] * filter_weight[w_col, w_col])
          result_arr.append(np.sum(result))

      # 연산 결과를 크기에 맞게 변경
      result_arr = np.array(result_arr).reshape(filter_shape, -1)

      filter_result_arr.append(result_arr)
    
    # 결과의 저장
    self.delta.append(filter_result_arr)
  
  def cal_cnn(self, input, target, filter_size, filter_count):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []

    # 가중치 행렬 생성
    weight_arr = []

    # 타겟값 저장
    self.target = target

    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):

      # 필터 크기에 맞는 임의의 가중치 생성
      weight = np.random.random(filter_size * filter_size).reshape(filter_size, filter_size)
      
      weight_arr.append(weight)
      
      # 합성곱 연산 결과가 저장된다.
      result_arr = []

      # 합성곱 연산 수행
      for col in range(input.shape[0] - weight.shape[0] + 1):
        for row in range(input.shape[1] - weight.shape[1] + 1):
          result = []
          for w_col in range(weight.shape[0]):
            for w_row in range(weight.shape[1]):
              result.append(input[col + w_col, row + w_row ] * weight[w_col, w_col])
          result_arr.append(np.sum(result))

      # 연산 결과를 크기에 맞게 바꿔준다.
      result_arr = np.array(result_arr).reshape(input.shape[0] - weight.shape[0] + 1, -1)

      # 연산 결과의 저장 (numpy 형태)
      filter_result_arr.append(result_arr)
    
    # 가중치 행렬의 저장
    self.cnn_weight.append(weight_arr)

    # 층에다가 결과를 저장한다. 모든 필터의 결과가 저장
    self.layer_result.append(filter_result_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  # 패딩 cnn 연산 수행
  def same_padding_cnn(self, input, filter_size, filter_count):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []
    
    # 데이터 크기 지정
    data_size = input.shape[0]

    # 가중치 행렬 생성
    weight_arr = []

    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):

      # 커널 크기에 맞는 임의의 가중치 생성
      weight = np.random.random(filter_size * filter_size).reshape(filter_size, filter_size)
      
      # 가중치의 저장
      weight_arr.append(weight)
      
      # 합성곱 연산 결과가 저장된다.
      result_arr = []

      # 합성곱을 시행할, 패딩을 수행한 입력값
      padding_matrix = self.pad.padding(input, int(data_size / 2))

      # 합성곱 연산 수행
      for col in range(input.shape[0]):
        for row in range(input.shape[1]):
          result = []
          for w_col in range(weight.shape[0]):
            for w_row in range(weight.shape[1]):
              result.append(padding_matrix[col + w_col, row + w_row ] * weight[w_col, w_row])
          result_arr.append(np.sum(result))

      # 연산 결과를 크기에 맞게 바꿔준다.
      result_arr = np.array(result_arr).reshape(data_size, -1)

      # 연산 결과의 저장
      filter_result_arr.append(result_arr)

    # 가중치의 저장
    self.cnn_weight.append(weight_arr)   
    
    # 층에다가 결과를 저장한다.
    self.layer_result.append(filter_result_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  # 두 번째 층 이후의 cnn 연산 함수, 입력값은 이전 층의 출력이 된다.
  def layer_cnn(self, filter_size, filter_count):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []

    # 각 layer 별 가중치를 저장할 리스트
    weight_arr = []

    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):
      # 필터를 저장할 배열
      filter_arr = []

      # 필터의 생성, 그 크기와 갯수는 이전 출력의 특성 맵의 크기와 동일
      for j in range(len(self.layer_result[-1])):
        # 필터 크기에 맞는 임의의 가중치 생성
        filter = np.random.random(filter_size * filter_size).reshape(filter_size, filter_size)

        # (n, n, 입력 특성 맵의 개수) 의 가중치, 필터가 filter_count 만큼 생성된다.
        filter_arr.append(filter)
      
      # 필터 배열의 저장
      weight_arr.append(filter_arr)
    
    # 128, 64, 3, (28,28)
    print(len(weight_arr), len(weight_arr[0]), len(weight_arr[0][0]), self.layer_result[-1][0].shape)

    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):
      print(i , "/", filter_count)
      
      # 채널별 필터의 연산 결괏값 행렬을 더한 합
      result_sum = np.zeros((self.layer_result[-1][0].shape[0], self.layer_result[-1][0].shape[0]))

      # 이전 출력의 채널 개수만큼 반복
      for j in range(len(self.layer_result[-1])):
        # 하나의 채널에 대한 (filter_size, filter_size, len(self.layer_result[-1])) 의 cnn 연산을 수행해야 함
        
        # 합성곱을 시행할 패딩을 수행한 입력값
        padding_matrix = self.pad.padding(self.layer_result[-1][j], int(self.layer_result[-1][j].shape[0] / 2))

        result_arr = []

        # 하나의 입력에 대한 cnn 연산 수행
        # 이전 출력의 모양에 따른 반복
        for col in range(self.layer_result[-1][j].shape[0]):
          for row in range(self.layer_result[-1][j].shape[1]):
            result = []
            for w_col in range(len(weight_arr[0][0])):
              for w_row in range(len(weight_arr[0][0])):
                result.append(padding_matrix[col + w_col, row + w_row ] * weight_arr[i][j][w_col, w_row])
            result_arr.append(np.sum(result))
          
        # 연산 결과를 크기에 맞게 바꿔준다.
        result_arr = np.array(result_arr).reshape(self.layer_result[-1][j].shape[0], -1)

        # 연산 결괏값 행렬을 모두 더해준다.
        result_sum = result_sum + result_arr
      
      filter_result_arr.append(result_sum)
    
    # 가중치의 저장
    self.cnn_weight.append(weight_arr)

    # 층에다가 결과를 저장한다.
    self.layer_result.append(filter_result_arr)

  def pooling(self, pooling_size):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []

    # 이전 레이어의 필터 개수만큼 반복.
    for i in range(len(self.layer_result[-1])):
      # numpy array 의 반환
      pooling_result = self.pool.max_pooling(self.layer_result[-1][i], pooling_size)

      filter_result_arr.append(pooling_result)
    
    self.layer_result.append(filter_result_arr)

  def flatten(self):
    data = self.layer_result[-1]
    
    data_arr = []

    # 필터의 개수만큼 반복
    for filter_count in range(len(data)):
      for filter_size_col in range(data[filter_count].shape[0]):
        for filter_size_row in range(data[filter_count].shape[1]):
          data_arr.append(data[filter_count][filter_size_col][filter_size_row])

    self.layer_result.append(np.array(data_arr).reshape(-1,1))

  # MLP 층의 연결, 순전파 연산

  def mlp_forward_cal(self, node_count):

    # MLP 계산의 결과를 저장
    self.layer_result.append(self.mlp.forward_cal(self.layer_result[-1], node_count))

    self.cnn_weight.append(self.mlp.weight[-1])

  # 출력층 MLP 에 대한 delta 연산
  def cal_delta_result_layer(self):

    # 마지막 레이어의 출력을 전달한다.
    self.delta.append(self.mlp.cal_delta_result(self.layer_result[-1], self.target))

  def cal_delta_hidden_layer(self):
    
    # 노드 입력값을 넘겨준다.
    self.delta.append(self.mlp.cal_delta_hidden())

  def cal_weight_update_mlp(self, learning_rate):
    self.mlp.weight_update(learning_rate)

  def cal_delta_pooling(self):
    n = 0

    # 새로 저장되는 delta 값의 크기 계산
    kernel_delta = np.zeros(((self.layer_result[self.layer - len(self.delta) - 2][0]).shape[0] ** 2) * len(self.layer_result[self.layer - len(self.delta) - 1])).reshape(-1,1)

    # 커널의 개수만큼의 반복, 풀링 후 커널의 개수
    for i in range(len(self.layer_result[self.layer - len(self.delta) - 1])):
      # 커널별 최댓값의 위치를 탐색한다.
      max_position = self.pool.matching_pooling_result(cnn.layer_result[self.layer - len(self.delta) - 2][0], cnn.layer_result[self.layer - len(self.delta) - 1][0])

      for j in range(len(max_position)):
        # 추가되는 값의 위치 계산
        delta_position = (i * (self.layer_result[self.layer - len(self.delta) - 2][i]).shape[0] ** 2)
        delta_position = delta_position + (max_position[j][0] * (self.layer_result[self.layer - len(self.delta) - 2][i]).shape[0]) + max_position[j][1]

        kernel_delta[delta_position][0] = self.delta[len(self.delta) - 1][n]
        
        n = n + 1

    self.delta.append(kernel_delta)
        

In [147]:
cnn = CNN()

In [148]:
cnn.target = target

In [149]:
cnn.same_padding_cnn(input, 7, 64)

In [150]:
cnn.pooling(2)

In [ ]:
cnn.layer_cnn(3, 128)

In [152]:
cnn.pooling(2)

In [ ]:
cnn.layer_cnn(3, 256)

In [154]:
cnn.pooling(2)

In [155]:
cnn.flatten()

In [156]:
print(cnn.layer_result[-2][0].shape, len(cnn.layer_result[-2]), cnn.layer_result[-1].shape)

(4, 4) 256 (4096, 1)


In [157]:
print(len(cnn.mlp.node_output), len(cnn.mlp.node_input), len(cnn.mlp.weight))

0 0 0


In [158]:
cnn.mlp_forward_cal(128)

In [159]:
cnn.mlp_forward_cal(64)

In [160]:
cnn.mlp_forward_cal(10)

In [161]:
cnn.cal_delta_result_layer()

In [162]:
cnn.cal_delta_hidden_layer()

In [163]:
cnn.cal_delta_hidden_layer()

In [164]:
cnn.cal_delta_hidden_layer()

In [165]:
cnn.cal_weight_update_mlp(0.01)